In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
import glob
import csv
import os
import sys

from datetime import datetime, timedelta
from tsmoothie.smoother import *
from tsmoothie.utils_func import sim_seasonal_data
from tsmoothie.smoother import DecomposeSmoother

In [2]:
sys.stdout = open('/app/script/water_ans/ans_files/error_log.txt','a')

In [3]:
#today = datetime.now()
today = datetime(2021, 1, 6)

In [4]:
last_date=today-timedelta(weeks=1)

In [5]:
today = today.strftime('%Y-%m-%d')#ex)2019-03-14

In [6]:
last_date = last_date.strftime('%Y-%m-%d')#ex)2019-03-14

In [7]:
import CUBRIDdb
import pandas as pd

conn = CUBRIDdb.connect('CUBRID:xxx.xx.xx.xxx:30000:dgsmwtanalysis:::','xxxxxx','xxxxxxx')
conn.set_autocommit(False)
cur = conn.cursor()

selectQry = """\
    SELECT
        cstmr_no,
        mrnr_no,
        calbr_code,
        smrt_trmnl_no,
        mtinsp_dt,
        smrt_mtinsp_hour_usgqty
    FROM tb_smrt_mtinsp_info
    WHERE creat_stle = 'N'
    AND mrnr_no = 'A00'
    AND mtinsp_dt BETWEEN TO_DATETIME('{last_date} 01:00:00', 'YYYY-MM-DD HH24:MI:SS') AND TO_DATETIME('{today} 00:00:00', 'YYYY-MM-DD HH24:MI:SS')
    AND SUBSTR(cstmr_no, 0,4) != '2007'
    ORDER BY cstmr_no, smrt_trmnl_no, mtinsp_dt
""".format(last_date=last_date, today=today)

selectQry = selectQry.replace("\n\t"," ")
selectQry = selectQry.replace("\n"," ") 

cur.execute(selectQry)
rows = cur.fetchall()

df = pd.DataFrame(rows)
df.columns=['cstmr_no', 'mrnr_no', 'calbr_code', 'smrt_trmnl_no', 'mtinsp_dt', 'smrt_mtinsp_hour_usgqty']

In [8]:
df_du = df.drop_duplicates(["cstmr_no"], keep="last")

In [9]:
try:
    for index, row in df_du.iterrows():
        lst = df[df['cstmr_no'] == row["cstmr_no"]]
        smoother = LowessSmoother(smooth_fraction=0.2, iterations=1)
        lst[['smrt_mtinsp_hour_usgqty']] = lst[['smrt_mtinsp_hour_usgqty']].apply(pd.to_numeric)
        smoother.smooth(lst['smrt_mtinsp_hour_usgqty'])
        low, up = smoother.get_intervals('prediction_interval')

        d1 = np.array(lst['smrt_mtinsp_hour_usgqty'])
        d2 = up[0]
        d3 = np.array(lst['cstmr_no'])
        d4 = np.array(lst['mrnr_no'])
        d5 = np.array(lst['calbr_code'])
        d6 = np.array(lst['smrt_trmnl_no'])
        d7 = np.array(lst['mtinsp_dt'])
        dt = pd.DataFrame(x for x in zip(d1,d2,d3,d4,d5,d6,d7))
        dt.columns = ['smrt_mtinsp_hour_usgqty', 'upper_val','cstmr_no', 'mrnr_no', 'calbr_code', 'smrt_trmnl_no', 'mtinsp_dt']

        dt['rate'] = dt['smrt_mtinsp_hour_usgqty'] / dt['upper_val'] * 100
        dt['cal'] = dt['smrt_mtinsp_hour_usgqty'] - dt['upper_val']

        
        if dt['calbr_code'][0] == '13' and dt['cal'].max() >= 1:
            insert_query = """\
                INSERT INTO TB_USGQTY_ABNRML_ANALS_TESST
                (
                    ANALS_STDR_DAY,
                    CSTMR_NO,
                    MRNR_NO,
                    CALBR_CODE,
                    EXCSS_USGQTY_EXCESS_RT,
                    FRST_REGIST_DT
                )
                VALUES
                (
                    DATE_FORMAT(SYSDATETIME, '%Y%m%d'),
                    '{cstmr_no}',
                    '{mrnr_no}',
                    '{calbr_code}',
                    {cal},
                    CURRENT_TIMESTAMP
                )
            """.format(cstmr_no=row['cstmr_no'], mrnr_no=row['mrnr_no'],calbr_code=row['calbr_code'], cal=dt['cal'].max())          
            insert_query = insert_query.replace("\n\t"," ")
            insert_query = insert_query.replace("\n"," ")  
            cur.execute(insert_query)

            for idx, val in dt.iterrows():
                insert_query = """\
                    INSERT INTO TB_USGQTY_ABNRML_GRAPH_TEST
                    (
                        CSTMR_NO,
                        MRNR_NO,
                        ANALS_STDR_DAY,
                        USGQTY_RECPTN_DT,
                        USGQTY,
                        UPPR_VALUE,
                        EXCESS_STDR_VALUE,
                        FRST_REGIST_DT
                    )
                    VALUES
                    (
                        '{cstmr_no}',
                        '{mrnr_no}',
                        DATE_FORMAT(SYSDATETIME, '%Y%m%d'),
                        DATE_FORMAT('{mtinsp_dt}', '%Y%m%d%H%i%s'),
                        '{smrt_mtinsp_hour_usgqty}',
                        {upper_val},
                        1,
                        CURRENT_TIMESTAMP
                    )
                """.format(cstmr_no=val['cstmr_no'], mrnr_no=val['mrnr_no'],mtinsp_dt=val['mtinsp_dt'], 
                           smrt_mtinsp_hour_usgqty=val['smrt_mtinsp_hour_usgqty'],upper_val=val['upper_val'])          
                insert_query = insert_query.replace("\n\t"," ")
                insert_query = insert_query.replace("\n"," ")  
                cur.execute(insert_query)
                
            
        elif dt['calbr_code'][0] != '13' and dt['rate'].max() > 300:
            insert_query = """\
                INSERT INTO TB_USGQTY_ABNRML_ANALS_TESST
                (
                    ANALS_STDR_DAY,
                    CSTMR_NO,
                    MRNR_NO,
                    CALBR_CODE,
                    EXCSS_USGQTY_EXCESS_RT,
                    FRST_REGIST_DT
                )
                VALUES
                (
                    DATE_FORMAT(SYSDATETIME, '%Y%m%d'),
                    '{cstmr_no}',
                    '{mrnr_no}',
                    '{calbr_code}',
                    {rate},
                    CURRENT_TIMESTAMP
                )
            """.format(cstmr_no=row['cstmr_no'], mrnr_no=row['mrnr_no'],calbr_code=row['calbr_code'], rate=dt['rate'].max())          
            insert_query = insert_query.replace("\n\t"," ")
            insert_query = insert_query.replace("\n"," ")  
            cur.execute(insert_query)

            for idx, val in dt.iterrows():
                insert_query = """\
                    INSERT INTO TB_USGQTY_ABNRML_GRAPH_TEST
                    (
                        CSTMR_NO,
                        MRNR_NO,
                        ANALS_STDR_DAY,
                        USGQTY_RECPTN_DT,
                        USGQTY,
                        UPPR_VALUE,
                        EXCESS_STDR_VALUE,
                        FRST_REGIST_DT
                    )
                    VALUES
                    (
                        '{cstmr_no}',
                        '{mrnr_no}',
                        DATE_FORMAT(SYSDATETIME, '%Y%m%d'),
                        DATE_FORMAT('{mtinsp_dt}', '%Y%m%d%H%i%s'),
                        '{smrt_mtinsp_hour_usgqty}',
                        {upper_val},
                        300,
                        CURRENT_TIMESTAMP
                    )
                """.format(cstmr_no=val['cstmr_no'], mrnr_no=val['mrnr_no'],mtinsp_dt=val['mtinsp_dt'], 
                           smrt_mtinsp_hour_usgqty=val['smrt_mtinsp_hour_usgqty'],upper_val=val['upper_val'])          
                insert_query = insert_query.replace("\n\t"," ")
                insert_query = insert_query.replace("\n"," ")  
                cur.execute(insert_query)
                
except Exception as e:
    print("사용량이상 ::: " + str(datetime.now()))
    print(e)
    conn.rollback()
else:
    conn.commit()

/usr/local/lib64/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.6/site-packages/tsmoothie/smoother.py:1144: RuntimeWarning: invalid value encountered in true_divide
  data = data / max_data
/usr/local/lib/python3.6/site-packages/tsmoothie/smoother.py:1140: RuntimeWarning: invalid value encountered in true_divide
  X = np.arange(timesteps) / (timesteps - 1)
/usr/local/lib/python3.6/site-packages/tsmoothie/regression_basis.py:187: RuntimeWarning: invalid value encountered in true_divide
  X_base = np.abs(X / h).clip(0.0, 1.0)
/usr/local/lib/python3.6/site-packages/tsmoothie/utils_func.py:299: RuntimeWarning: divide by zero encountered in true_divide
  mse 